In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import multiprocessing
import pickle
import os
import psutil
import gc
import random
from PF_Aso1_0_CPU import *

In [27]:
!python Run_Aso1_0_CPU10006.py

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 37596.84it/s]
1 0.1390838623046875
2 0.13895797729492188
3.2022480964660645
  0%|                                                    | 0/10 [00:00<?, ?it/s]3 0.1392669677734375
4 0.13559341430664062
5 0.13607406616210938
6 0.136077880859375
 10%|████▍                                       | 1/10 [00:03<00:30,  3.44s/it]3 0.13611221313476562
4 0.1358795166015625
5 0.13610458374023438
6 0.13610458374023438
 20%|████████▊                                   | 2/10 [00:06<00:27,  3.38s/it]3 0.13612747192382812
4 0.13615798950195312
5 0.13623809814453125
6 0.13623809814453125
 30%|█████████████▏                              | 3/10 [00:10<00:23,  3.34s/it]3 0.13627243041992188
4 0.13620376586914062
5 0.13628768920898438
6 0.13628768920898438
 40%|█████████████████▌                          | 4/10 [00:13<00:20,  3.35s/it]3 0.13631439208984375
4 0.13629531860351562
5 0.13656997680664062
6 0.13656234741210938
 50%|████████████████

In [44]:
outputdir_2 = outputdir + '2/'
with open(outputdir_2 + 'w_6.pkl', 'rb') as f:
    θ_6 = pickle.load(f)
θ_6[1]

0.002135674111055924

In [43]:
outputdir_1 = outputdir + '1/'
with open(outputdir_1 + 'w_6.pkl', 'rb') as f:
    θ_6 = pickle.load(f)
θ_6[1]

0.002135674111055924

In [3]:
seed = 6

obs_series = pd.read_csv(datadir + 'data.csv', delimiter=',')
obs_series = np.array(obs_series.iloc[:,1:]).T

T = obs_series.shape[1]
N = 1_0
Λ_scale = 1.0
cd_scale = 1.0

case = 'actual data, seed = ' + str(seed) + ', T = ' + str(T) + ', N = ' + str(N) + ', Λ_scale = ' + str(Λ_scale) + ', cd_scale = ' + str(cd_scale)
try: 
    casedir = outputdir + case  + '/'
    os.mkdir(casedir)
except:
    casedir = outputdir + case  + '/'

In [22]:
if __name__ == '__main__':
    np.random.seed(seed)
    start_time = time.time()

    D_0 = obs_series[:,[0]]

    Input_0 = [[D_0, Λ_scale, cd_scale, seed+i] for i in range(N)]
    pool = multiprocessing.Pool()
    Output_0 = pool.map(init, tqdm(Input_0))
    θ_t_particle = [i[0] for i in Output_0]
    X_t_particle = [i[1] for i in Output_0]
    H_t_particle = [i[2] for i in Output_0]
    
    run_time = time.time() - start_time
    print(run_time)    
    for t in tqdm(range(1)):

        D_t_next = obs_series[:,[t+1]]

        Input = [[D_t_next, X_t_particle[i], H_t_particle[i], seed+t+i] for i in range(N)]
        pool = multiprocessing.Pool()
        Output = pool.map(recursive, Input)
        θ_t_next_particle = [i[0] for i in Output]
        X_t_next_particle = [i[1] for i in Output]
        H_t_next_particle = [i[2] for i in Output]
        ν_t_next_particle = [i[3] for i in Output]    

        with open(casedir + 'θ_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(θ_t_next_particle, f)

        # with open(casedir + 'X_' + str(t+1) + '.pkl', 'wb') as f:
        #     pickle.dump(X_t_next_particle, f)
        # with open(casedir + 'H_' + str(t+1) + '.pkl', 'wb') as f:
        #     pickle.dump(H_t_next_particle, f)

        w_t_next = ν_t_next_particle/np.sum(ν_t_next_particle)

        try:
            count_all = sp.stats.multinomial.rvs(N, w_t_next)
        except:
            for i in range(w_t_next.shape[0]):
                if w_t_next[i]>(np.sum(w_t_next[:-1]) - 1):
                    w_t_next[i] = w_t_next[i] - (np.sum(w_t_next[:-1]) - 1)
                    break
            count_all = sp.stats.multinomial.rvs(N, w_t_next)

        with open(casedir + 'w_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(w_t_next, f)

        with open(casedir + 'count_' + str(t+1) + '.pkl', 'wb') as f:
            pickle.dump(count_all, f)

        X_t_particle = []
        H_t_particle = []

        for i in range(N):
            if count_all[i] != 0:
                for n in range(count_all[i]):
                    X_t_particle.append(X_t_next_particle[i])
                    H_t_particle.append(H_t_next_particle[i])



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2712.83it/s]


3.1981120109558105


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.90s/it]
